In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from multiprocessing import Pool, Process
from haversine import haversine
import os

In [2]:
df = pd.read_csv('/home/mskim/Project/MN_soft/Data/201904_link_01.txt')

In [3]:
df

,lon,lat,index,point_in_time,od_id,seq,flink,tlink,turn,flink_len,...,track_in_date,track_in_time,track_speed,track_travel_time,track_sum_travel_time,static_spd,hq_speed,ol_rot_spd,ol_spd,svc_name
0,127.076500,37.502833,0,53936,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
1,127.076278,37.502861,1,53937,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
2,127.076028,37.502889,2,53938,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
3,127.075806,37.502889,3,53939,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
4,127.075583,37.502917,4,53940,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
5,127.075361,37.502917,5,53942,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
6,127.075139,37.502944,6,53943,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
7,127.074889,37.502944,7,53944,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
8,127.074667,37.502972,8,53945,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0
9,127.074444,37.503028,9,53947,20190403B00003468,0,2029169,2029166,3,5420,...,20190403,53936,590,331,331,42,0,54,54,0


In [4]:
df = df[['lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
             'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
             'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
             'ol_rot_spd', 'ol_spd', 'svc_name']]

In [3]:
first_index = df[(df['index']== 0) & (df['seq'] == 0)].index
first_index

Int64Index([       0,     1130,     2221,     3195,     3969,     4205,
                4547,     4897,     5765,     5924,
            ...
            51510501, 51510994, 51511350, 51511732, 51512290, 51512605,
            51513048, 51513449, 51513559, 51514033],
           dtype='int64', length=130356)

In [4]:
def Make_id_parser(data):
    if not os.path.isdir('./Data/id_based_parsing_msedit'):
        os.mkdir('./Data/id_based_parsing_msedit')
    for i in range (len(data)):
        id_in = df.loc[data[i], 'od_id']
        if(i != len(data)-1):
            file_out = df.loc[data[i]:(data[i+1]-1)]
            file_out.to_csv('./Data/id_based_parsing_msedit/id_{}.csv'.format(id_in))

In [5]:
Make_id_parser(first_index)

In [12]:
def Make_preprocess(folder_name, start, end, num_file, IDs, calc_time,divide_num):
    # make directory
    if not os.path.isdir('./Data/'+folder_name):
        os.mkdir('./Data/'+folder_name)
    
    f =open("./Data/"+folder_name+"/data_{:02d}".format(num_file+1), 'a')
    for count in range(start, end):
    #if not exist file, skip
        if not os.path.isfile('./Data/id_based_parsing_msedit/id_{}.csv'.format(IDs[count])):
            print('id_{}.csv can not be found'.format(IDs[count]))
            count = count + 1
            continue
    #read id file            
        df1 = pd.read_csv('./Data/id_based_parsing_msedit/id_{}.csv'.format(IDs[count]))
        df1 = df1[df1.index % divide_num ==0]
        print(count, IDs[count])
        df2 = df1.loc[0]

    # lngs, lats
        df2['lngs'] = df1['lon'].values
        df2['lats'] = df1['lat'].values

    # time, dist
        tot_time = df1.loc[df1.index[-1], 'track_sum_travel_time']
        tot_dist = df1.loc[df1.index[-1], 'sum_len']
        df2['time'] = tot_time / 10
        df2['dist'] = tot_dist / 10000

    # to datetime
        time = df1.loc[0, 'track_in_time']
        date = df1.loc[0, "track_in_date"]
        test = str(date) + str('%6d' %time)
        test = test.replace(' ', '0')
        test = pd.to_datetime(test)

    # weekID, dateID, timeID
        weekID = test.weekday()
        df2['weekID'] = weekID
        df2['dateID'] = test.day
        timeID = (test.hour * 60) + (test.minute) + int(test.second / 60)
        df2['timeID'] = timeID

    # driverID
        df2 = df2.rename({'od_id': 'driverID'})
        driver =df2['driverID']
        driver = driver.replace('B', '')
        driver = driver[4:]
        df2['driverID'] = int(driver)
        
    # calc_time
        df2['calc_time'] = calc_time[count]

    # dist gap
        distgap = (df1['sum_len'] - df1['flink_len'])/10000
        dist = []
        for i in range(len(df1['lon'])):
            if(i < len(df1['lon']) - 1):
                dist.append(haversine((df1.loc[i*divide_num,'lon'], df1.loc[i*divide_num, 'lat']), (df1.loc[(i+1)*divide_num, 'lon'], df1.loc[(i+1)*divide_num, 'lat'])))
        dist = pd.Series(dist)
        flink_index = np.array(df1.index[df1['index'] == 0])
        lis = np.array(df1.index)
        idx_del = np.delete(lis, flink_index)
        for i in idx_del:
            distgap[i] = distgap[i-1*divide_num] +dist[i/divide_num-1]
#             distgap[i] = distgap[i-1] +dist[i-1]

    # time gap
        hour = (df1['point_in_time'] / 10000).astype(int)
        minute = (df1['point_in_time'] / 100).astype(int) - hour * 100
        second = df1['point_in_time'].astype(int) - minute * 100 - hour * 10000
        timegap = hour * 60 * 60 + minute * 60 + second
        timegap = timegap - timegap[0]
        timegap = timegap.astype(float)
        df2['time_gap'] = timegap.values
        df2['dist_gap'] = distgap.values

    # states == 1.0
        states = [1.0] * len(df2['lngs'])
        df2['states'] = states

    # drop columns
        df2 = df2.drop(['Unnamed: 0', 'lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
             'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
             'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
             'ol_rot_spd', 'ol_spd', 'svc_name'])
    #write file
        f.write(df2.to_json())
        f.write('\n')
        count = count + 1
    f.close()
    print('file was made')


In [13]:
import warnings

warnings.filterwarnings(action='ignore') 


In [14]:
if __name__ =='__main__':
#     df_link = pd.read_csv('./Data/201904_link_01.txt')
#     df_link = df_link[['od_id', 'index', 'seq', 'lon', 'lat',
#          'sum_len', 'track_in_date', 'track_in_time', 'track_sum_travel_time', 'calc_travel_time']]
    df = pd.read_csv('./Data/4_summary.txt')
    IDs = df['od_id'].values   
    calc_time = df['calc_travel_time'].values
    
    divide_num = 1    ##  divide_num*20 
    
    folder_name = 'preprocessing_'+str(divide_num*20)+'_id_last_4'
    
    num_IDs = len(IDs)
    IDs_in_onefile = 10000
    num_file = int(num_IDs / IDs_in_onefile)+1

    procs=[]
    index = []
    for i in range(0,num_file):
        if IDs_in_onefile*(i+1) <=num_IDs:
            index = index+[[i*IDs_in_onefile, IDs_in_onefile*(i+1)]]
        else : 
            index = index+[[i*IDs_in_onefile, num_IDs]]
#     index = [[10001,20000],[20001,30000],[30001,40000],[40001,50000]]
    for i, number in enumerate(index):
        proc = Process(target=Make_preprocess, args=(folder_name, number[0], number[1], 
                                                     i, IDs, calc_time, divide_num))
        procs.append(proc)
        proc.start()
    print("End of Preprocessing ")
    print("End of Preprocessing ")
    print("End of Preprocessing ")
    print("End of Preprocessing ")

End of Preprocessing 
End of Preprocessing 
End of Preprocessing 
End of Preprocessing 
10000 20190421B03843765
20000 20190406B00183011
0 20190425B01688433
30000 20190406B02387793
40000 20190410B00570015
80000 20190429B02135604
60000 20190407B01788818
50000 20190404B00906704
100000 20190427B02239326
110000 20190422B00826516
70000 20190414B03649131
90000 20190426B01380257
120000 20190406B04304075
10001 20190415B00712332
20001 20190406B00183492
110001 20190422B00826825
100001 20190427B02239635
30001 20190406B02387809
1 20190425B01689654
80001 20190429B02136235
60001 20190407B01789287
120001 20190406B04304443
70001 20190414B03649720
40001 20190410B00572277
90001 20190426B01380320
10002 20190415B00712857
50001 20190404B00907245
110002 20190422B00827786
100002 20190427B02240451
20002 20190406B00183661
80002 20190429B02136743
120002 20190406B04304666
60002 20190407B01789358
30002 20190406B02388443
70002 20190414B03650191
2 20190425B01690044
40002 20190410B00575721
90002 20190426B01380926
100

50025 20190404B00925118
120028 20190406B04332289
30027 20190406B02410614
90026 20190426B01394665
40024 20190410B00594528
26 20190425B01701446
10027 20190415B00738289
80026 20190429B02155906
50026 20190404B00926087
110027 20190422B00840213
60028 20190407B01813075
100024 20190427B02254036
20025 20190406B00204121
120029 20190406B04332840
30028 20190406B02413088
40025 20190410B00596223
90027 20190426B01395157
80027 20190429B02156223
70026 20190414B03666983
10028 20190415B00739545
27 20190425B01701595
110028 20190422B00840869
50027 20190404B00926158
100025 20190427B02255260
20026 20190406B00204163
120030 20190406B04333598
30029 20190406B02418911
60029 20190407B01813810
90028 20190426B01395474
70027 20190414B03668164
40026 20190410B00596345
80028 20190429B02156816
10029 20190415B00740609
28 20190425B01701782
110029 20190422B00842279
50028 20190404B00926238
100026 20190427B02255273
30030 20190406B02418917
90029 20190426B01395618
120031 20190406B04334910
20027 20190406B00204390
60030 20190407B

120055 20190406B04368719
51 20190425B01713395
50054 20190404B00939912
90053 20190426B01417871
40048 20190410B00611591
30055 20190406B02446460
60053 20190407B01844043
10053 20190415B00767006
80054 20190429B02176912
110053 20190422B00853256
70051 20190414B03696823
20050 20190406B00234648
100051 20190427B02282596
120056 20190406B04368816
50055 20190404B00940838
52 20190425B01714922
90054 20190426B01420968
30056 20190406B02448926
60054 20190407B01844674
10054 20190415B00771269
80055 20190429B02177573
40049 20190410B00613089
110054 20190422B00854437
70052 20190414B03699568
20051 20190406B00234712
50056 20190404B00941655
90055 20190426B01421882
100052 20190427B02283250
53 20190425B01715607
120057 20190406B04369945
30057 20190406B02448989
80056 20190429B02178583
60055 20190407B01844688
10055 20190415B00771320
40050 20190410B00613122
110055 20190422B00856343
70053 20190414B03699838
20052 20190406B00236421
50057 20190404B00942562
90056 20190426B01422055
100053 20190427B02283388
120058 20190406B

77 20190425B01737994
50082 20190404B00952687
80080 20190429B02194371
100076 20190427B02313060
90081 20190426B01455356
110080 20190422B00883816
40072 20190410B00629390
60079 20190407B01867143
120083 20190406B04405637
70074 20190414B03714941
10079 20190415B00788076
20077 20190406B00260409
50083 20190404B00952940
30083 20190413B01640095
80081 20190429B02194952
100077 20190427B02315248
110081 20190422B00883864
90082 20190426B01457084
78 20190425B01738197
120084 20190406B04405697
60080 20190407B01867590
70075 20190414B03717706
40073 20190410B00630544
10080 20190415B00788803
20078 20190406B00260772
50084 20190404B00953009
30084 20190413B01640424
100078 20190427B02315804
90083 20190426B01457549
110082 20190422B00885726
80082 20190429B02195348
60081 20190407B01869709
70076 20190414B03718047
79 20190425B01738199
120085 20190406B04407870
10081 20190415B00788877
40074 20190410B00631548
50085 20190404B00954292
20079 20190406B00262061
30085 20190413B01642225
90084 20190426B01457844
110083 20190422B

110107 20190422B00904599
102 20190425B01752842
20102 20190406B00292462
90108 20190426B01477933
30110 20190413B01670346
80106 20190429B02216553
60104 20190407B01893280
100103 20190427B02329448
40096 20190410B00647717
70101 20190414B03739204
120110 20190406B04436007
103 20190425B01753136
110108 20190422B00905195
50111 20190404B00965830
20103 20190406B00292520
10105 20190415B00808485
30111 20190413B01671055
80107 20190429B02216690
90109 20190426B01478193
60105 20190407B01893433
100104 20190427B02330866
40097 20190410B00648107
70102 20190414B03739722
104 20190425B01753556
110109 20190422B00905407
120111 20190406B04436931
50112 20190404B00965913
10106 20190415B00809034
20104 20190406B00292536
80108 20190429B02218219
30112 20190413B01672520
90110 20190426B01479392
60106 20190407B01894345
40098 20190410B00648719
70103 20190414B03743713
120112 20190406B04438480
20105 20190406B00292676
10107 20190415B00809905
110110 20190422B00906821
50113 20190404B00966296
105 20190425B01753572
100105 20190427

...

32172 20190414B00403806
52097 20190404B01146661
62168 20190408B00277790
2091 20190423B00701905
12161 20190420B04544466
122169 20190406B04754515
42185 20190417B00006855
82153 20190413B05647215
112100 20190404B02979959
52098 20190404B01147256
2092 20190423B00703200
72117 20190415B01749018
22175 20190413B00554685
92168 20190430B00414746
122170 20190406B04754992
62169 20190408B00277923
32173 20190414B00404973
42186 20190417B00007436
12162 20190420B04544572
82154 20190413B05650406
102156 20190430B02049344
112101 20190404B02980602
92169 20190430B00416996
2093 20190409B00000817
72118 20190415B01750241
122171 20190406B04755699
102157 20190430B02049889
52099 20190404B01147664
42187 20190417B00008417
92170 20190430B00417633
32174 20190414B00405722
82155 20190413B05652423
62170 20190408B00278372
12163 20190420B04546207
112102 20190404B02981384
22176 20190413B00555137
2094 20190409B00002611
122172 20190406B04755708
42188 20190417B00008977
92171 20190430B00418791
72119 20190415B01752289
32175 20190

32199 20190414B00428001
102180 20190430B02063635
42212 20190417B00026751
112128 20190404B03004494
72143 20190415B01765416
2116 20190409B00021623
62193 20190408B00297050
12185 20190420B04572997
82177 20190413B05676087
12186 20190420B04573323
122198 20190406B04792730
92195 20190430B00430812
72144 20190415B01766172
52125 20190404B01169371
112129 20190404B03004886
32200 20190414B00428468
22201 20190413B00589207
102181 20190430B02064319
62194 20190408B00297095
2117 20190409B00022008
12187 20190420B04577640
42213 20190417B00026957
22202 20190413B00589966
82178 20190413B05676527
92196 20190430B00430934
52126 20190404B01170050
122199 20190406B04794193
72145 20190415B01766591
2118 20190409B00022636
42214 20190417B00027098
32201 20190414B00428971
112130 20190404B03006745
52127 20190404B01170474
22203 20190413B00590380
102182 20190430B02064835
62195 20190408B00297108
82179 20190413B05677026
92197 20190430B00431268
12188 20190420B04581710
122200 20190406B04795112
112131 20190404B03007239
32202 201

2141 20190409B00038158
62218 20190408B00308373
12212 20190420B04610430
42239 20190417B00041134
52151 20190404B01186590
22227 20190413B00628622
62219 20190408B00309151
102209 20190430B02084440
122223 20190406B04813517
92218 20190430B00444708
72169 20190415B01781237
112154 20190404B03023369
2142 20190409B00039288
52152 20190404B01186627
32226 20190414B00452174
82205 20190413B05708174
42240 20190417B00041167
122224 20190406B04813836
12213 20190420B04610930
92219 20190430B00445873
32227 20190414B00452323
112155 20190404B03023924
72170 20190415B01783255
62220 20190408B00309212
22228 20190413B00629055
42241 20190417B00041440
92220 20190430B00446043
52153 20190404B01187909
102210 20190430B02087851
2143 20190409B00039344
22229 20190413B00629102
82206 20190413B05709623
12214 20190420B04611543
32228 20190414B00453422
62221 20190408B00309257
122225 20190406B04814111
112156 20190404B03024675
92221 20190430B00446974
72171 20190415B01784466
42242 20190417B00041690
52154 20190404B01188206
22230 20190

112178 20190404B03041747
62245 20190408B00325924
72195 20190415B01806135
42265 20190417B00053267
82230 20190413B05750151
32253 20190414B00490431
92245 20190430B00457718
12238 20190420B04635654
52177 20190404B01203152
102236 20190430B02102290
22253 20190413B00642836
2169 20190409B00064684
122248 20190406B04834938
112179 20190404B03042769
62246 20190408B00327537
82231 20190413B05750969
22254 20190413B00642858
122249 20190406B04835388
72196 20190415B01806349
42266 20190417B00053984
12239 20190420B04636855
52178 20190404B01204073
92246 20190430B00457753
2170 20190409B00064801
32254 20190414B00492959
102237 20190430B02102408
62247 20190408B00327635
12240 20190420B04638806
112180 20190404B03043637
52179 20190404B01205335
32255 20190414B00494030
22255 20190413B00643376
2171 20190409B00065010
122250 20190406B04836234
102238 20190430B02102705
72197 20190415B01806476
92247 20190430B00457975
12241 20190420B04639132
42267 20190417B00054180
82232 20190413B05752357
62248 20190408B00328793
112181 201

122273 20190406B04862806
2195 20190409B00082581
62272 20190408B00345929
22279 20190413B00659536
42292 20190417B00068131


In [ ]:
2019042003795622

9223372036854775807

In [135]:
print("End")

End


In [115]:
df = pd.read_csv('./Data/4_summary.txt')
IDs = df['od_id'].values   
calc_time = df['calc_travel_time'].values


In [116]:
calc_time

array([ 1130, 33470, 43130, ..., 11760,  7200,  2090])

In [117]:
num_IDs = len(IDs)
IDs_in_onefile = 10000
num_file = int(num_IDs / IDs_in_onefile)+1

procs=[]
index = []
for i in range(0,num_file):
    if IDs_in_onefile*(i+1) <=num_IDs:
        index = index+[[i*IDs_in_onefile, IDs_in_onefile*(i+1)]]
    else : 
        index = index+[[i*IDs_in_onefile, num_IDs]]

In [118]:
print("num_IDs : ", len(IDs))
print("num_file : ", num_file)
print("index : ", index)

num_IDs :  125758
num_file :  13
index :  [[0, 10000], [10000, 20000], [20000, 30000], [30000, 40000], [40000, 50000], [50000, 60000], [60000, 70000], [70000, 80000], [80000, 90000], [90000, 100000], [100000, 110000], [110000, 120000], [120000, 125758]]


In [11]:
id = '20190406B00183011'
print(len(id))
print(id[4:])

17
0406B00183011


In [120]:
f =open("./Data/preprocess_temp/preprocessing_{}.txt".format(num_file), 'a')
# for count in range(0, 1000):
#if not exist file, skip
# if not os.path.isfile('./Data/id_based_parsing_msedit/id_{}.csv'.format('20190406B00183011')):
#     print('id_{}.csv can not be found'.format(IDs[count]))
#     count = count + 1
#     continue
#read id file            
df1 = pd.read_csv('./Data/id_based_parsing_msedit/id_{}.csv'.format(IDs[count]))
df1 = df1[df1.index % divide_num ==0]
print(count, IDs[count])
df2 = df1.loc[0]


1 20190425B01689654


In [121]:

# lngs, lats
df2['lngs'] = df1['lon'].values
df2['lats'] = df1['lat'].values

# time, dist
tot_time = df1.loc[df1.index[-1], 'track_sum_travel_time']
tot_dist = df1.loc[df1.index[-1], 'sum_len']
df2['time'] = tot_time / 10
df2['dist'] = tot_dist / 10000

# to datetime
time = df1.loc[0, 'track_in_time']
date = df1.loc[0, "track_in_date"]
test = str(date) + str('%6d' %time)
test = test.replace(' ', '0')
test = pd.to_datetime(test)

# weekID, dateID, timeID
weekID = test.weekday()
df2['weekID'] = weekID
df2['dateID'] = test.day
timeID = (test.hour * 60) + (test.minute) + int(test.second / 60)
df2['timeID'] = timeID

# driverID
df2 = df2.rename({'od_id': 'driverID'})
driver =df2['driverID']
driver = driver.replace('B', '')
driver = driver[12:]
df2['driverID'] = int(driver)

# calc_time
df2['calc_time'] = calc_time[count]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/py

In [122]:

# dist gap
distgap = (df1['sum_len'] - df1['flink_len'])/10000
dist = []


In [123]:
df1

,Unnamed: 0,lon,lat,index,point_in_time,od_id,seq,flink,tlink,turn,...,track_in_date,track_in_time,track_speed,track_travel_time,track_sum_travel_time,static_spd,hq_speed,ol_rot_spd,ol_spd,svc_name
0,41203127,127.038500,37.475528,0,183632,20190425B01689654,0,2023146,2023145,3,...,20190425,183632,58,1393,1393,6,6,4,4,0
5,41203132,127.038639,37.476472,5,183735,20190425B01689654,0,2023146,2023145,3,...,20190425,183632,58,1393,1393,6,6,4,4,0
10,41203137,127.038667,37.477389,10,183838,20190425B01689654,0,2023146,2023145,3,...,20190425,183632,58,1393,1393,6,6,4,4,0
15,41203142,127.038667,37.478083,3,183858,20190425B01689654,1,2023145,2021046,3,...,20190425,183851,277,77,1470,19,6,10,10,0
20,41203147,127.038556,37.478833,4,183906,20190425B01689654,2,2021046,2010340,3,...,20190425,183859,385,135,1605,17,9,9,9,0
25,41203152,127.038361,37.479556,1,183915,20190425B01689654,3,2010340,2009719,3,...,20190425,183912,227,276,1881,9,9,4,5,0
30,41203157,127.037889,37.480417,6,183930,20190425B01689654,3,2010340,2009719,3,...,20190425,183912,227,276,1881,9,9,4,5,0
35,41203162,127.037556,37.481000,1,183944,20190425B01689654,4,2009719,2021610,3,...,20190425,183940,167,205,2086,6,9,10,10,0
40,41203167,127.037083,37.481556,0,184000,20190425B01689654,5,2021610,2021613,3,...,20190425,184000,58,723,2809,7,9,5,5,0
45,41203172,127.036361,37.482278,5,184100,20190425B01689654,5,2021610,2021613,3,...,20190425,184000,58,723,2809,7,9,5,5,0


In [124]:
print(df1.loc[0,'lon'])
print( df1.loc[0, 'lat'])
print(df1.loc[5,'lon'])
print( df1.loc[5, 'lat'])

127.0385
37.475527777777785
127.03863888888893
37.47647222222222


In [125]:
for i in range(len(df1['lon'])):
    print(i)
    if(i < len(df1['lon']) - 1):
        dist.append(haversine((df1.loc[i*5,'lon'], df1.loc[i*5, 'lat']), (df1.loc[(i+1)*5, 'lon'], df1.loc[(i+1)*5, 'lat'])))



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


In [126]:
dist = pd.Series(dist)
flink_index = np.array(df1.index[df1['index'] == 0])
lis = np.array(df1.index)
idx_del = np.delete(lis, flink_index)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  after removing the cwd from sys.path.


In [127]:
idx_del

array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130,
       135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195,
       205, 210, 215, 220, 225, 230, 235, 240, 245, 255, 260, 265, 270,
       275, 280, 285, 290, 295, 305, 310, 315, 320, 325, 330, 335, 340,
       345, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410,
       415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475,
       480, 485, 490, 495, 505, 510, 515, 520, 530, 535, 540, 545, 550,
       555])

In [128]:
distgap

0      0.000
5      0.000
10     0.000
15     0.224
20     0.283
25     0.427
30     0.427
35     0.601
40     0.696
45     0.696
50     0.843
55     0.843
60     0.982
65     1.043
70     1.108
75     1.133
80     1.203
85     1.228
90     1.283
95     1.348
100    1.451
105    1.507
110    1.526
115    1.591
120    1.645
125    1.708
130    1.770
135    1.836
140    1.944
145    1.944
       ...  
410    5.651
415    5.710
420    5.782
425    5.782
430    5.896
435    5.958
440    6.049
445    6.077
450    6.107
455    6.191
460    6.332
465    6.413
470    6.462
475    6.549
480    6.549
485    6.651
490    6.741
495    6.769
500    6.867
505    6.896
510    6.993
515    6.993
520    7.095
525    7.160
530    7.224
535    7.288
540    7.399
545    7.445
550    7.518
555    7.578
Length: 112, dtype: float64

In [129]:
dist

0      0.065116
1      0.061475
2      0.046513
3      0.051731
4      0.052985
5      0.077998
6      0.053854
7      0.064356
8      0.093750
9      0.042339
10     0.096409
11     0.037878
12     0.073869
13     0.043269
14     0.048357
15     0.036789
16     0.019342
17     0.054636
18     0.046444
19     0.044967
20     0.040133
21     0.049801
22     0.041615
23     0.055215
24     0.044966
25     0.044966
26     0.046442
27     0.049800
28     0.073972
29     0.053189
         ...   
81     0.043520
82     0.053191
83     0.041614
84     0.100575
85     0.046666
86     0.055113
87     0.044967
88     0.072225
89     0.097542
90     0.044966
91     0.056103
92     0.097542
93     0.081675
94     0.081675
95     0.119477
96     0.075265
97     0.035763
98     0.042789
99     0.003721
100    0.040928
101    0.022325
102    0.122513
103    0.072225
104    0.078309
105    0.078640
106    0.043130
107    0.033450
108    0.078640
109    0.087757
110    0.094163
Length: 111, dtype: floa

In [130]:
for i in idx_del:
    print(i)
    distgap[i] = distgap[i-1*5] +dist[i/5-1]

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
205
210
215
220
225
230
235
240
245
255
260
265
270
275
280
285
290
295
305
310
315
320
325
330
335
340
345
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
505
510
515
520
530
535
540
545
550
555


In [131]:

# time gap
hour = (df1['point_in_time'] / 10000).astype(int)
minute = (df1['point_in_time'] / 100).astype(int) - hour * 100
second = df1['point_in_time'].astype(int) - minute * 100 - hour * 10000
timegap = hour * 60 * 60 + minute * 60 + second
timegap = timegap - timegap[0]
timegap = timegap.astype(float)
df2['time_gap'] = timegap.values
df2['dist_gap'] = distgap.values


In [132]:

# states == 1.0
states = [1.0] * len(df2['lngs'])
df2['states'] = states

# drop columns
df2 = df2.drop(['Unnamed: 0', 'lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
     'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
     'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
     'ol_rot_spd', 'ol_spd', 'svc_name'])
#write file
f.write(df2.to_json())
f.write('\n')
count = count + 1
f.close()
print('file was made')

file was made


In [112]:
df = pd.DataFrame(np.arange(10, 22).reshape(3, 4),
                  index=["a", "b", "c"],
                  columns=["0", "1", "2", "3"])

In [37]:
df.loc["a"]

A    10
B    11
C    12
D    13
Name: a, dtype: int64

In [ ]:
dist.set_index(pd.Index([5*i for i in range(len(distgap)+1)]))

In [7]:
test=3
print("{:03d}".format(test))

003


In [8]:
df = pd.read_csv('./DeepTTE/data/data_01')

ParserError: Error tokenizing data. C error: Expected 497 fields in line 3, saw 707
